In [2]:
import os
import math
import random #to set the seed to replicate results
from datetime import datetime,timedelta #for today's date
from dateutil.relativedelta import relativedelta
import sys

import psycopg2
import psycopg2.extras
from psycopg2.extensions import AsIs

import pandas as pd 
import numpy as np #for the e_logarithmic filter (and also some other mathematical operations)
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler #for preprocessing, it scales features using statistics that are robust to outliers.
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress #for the slope and the value of Y at X=0 of the linear trend line
from scipy.optimize import curve_fit
import tsmoothie #for the Kalman filter, it is an efficient recursive filter that evaluates the state of a dynamic system starting from a series of measurements subject to noise.

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM #the two main layers of the model
from tensorflow.keras import optimizers #for the training of the model
from tensorflow.keras.models import load_model

random.seed(42)  #set the seed to replicate results

print(tf.config.list_physical_devices('GPU'))

2024-01-16 09:18:55.943620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 09:18:55.943669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 09:18:55.944999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 09:18:55.953317: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 09:18:57.306233: W tensorflow/compiler/tf2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-01-16 09:18:58.041433: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 09:18:58.126332: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 09:18:58.126639: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

#### Functions 

In [3]:
'''Function for making sequences (blocks) of test and train data'''
def building_data_sequences(data_X,data_Y, timesteps): #timesteps means how many days we consider for each block

    X=[]
    y_MPNxP = []
    for i in range(len(data_X)-timesteps+1):  #how it works: every timesteps (e.g. 10 days) a block is constituted and for each block data and true values are stored
        X.append(data_X[i:(i+timesteps),:])
        y_MPNxP.append(data_Y[i+timesteps-1])
    return np.array(X), [np.array(y_MPNxP)]

'''Function for computing the analytical parameters'''
def sir_parameters(x,y): #sir stands for slope, intercept, rvalue (actually there's also the average trend line distance or avg_tld, but it came later)

  analytical_params = linregress(x, y)
  slope = analytical_params.slope
  intercept = analytical_params.intercept
  rvalue = analytical_params.rvalue #pay attention that here we have the correlaton coefficient (so not r2 that is the coefficient of determination)
  x_trend_line = slope*x + intercept #this is computed just for the avg_tld
  avg_trend_line_distance = np.mean(np.abs(x_trend_line-y))
  return slope,intercept,rvalue**2,avg_trend_line_distance

In [4]:
'''This is the timestep which indicates the window size'''
model_case_version_time_steps= '10'
'''This is the number of periods defined in the target(MPNxP) where x is the number of periods'''
model_case_version_main_target_code='5'

#today = '20220706'
today = datetime.today().strftime('%Y%m%d') #just for names of files (for now)
print('Today is', today)

Today is 20240116


In [5]:
''' Here, we define the case name'''
case = 'TSLA'
print(case)

TSLA


In [6]:
'''Here, we define the list of targets we are going to work on and also the average for each target (this value is used during the training for normalization\rescaling of some analytical parameters)'''
targets =['MPN'+model_case_version_main_target_code+'P']  #this must be changed whenever tha targets change
avg_prices_list = []

In [7]:
targets

['MPN5P']

In [8]:
#1L = dispersion = 1
#4L = slope + intercept + resqr + dispersion = 1
#5L = all the weighting = 1
#new paramaters case = intercept + slope + end intercept + correlation + dispersion
slope_weighting_exponent_ratio = 1
intercept_weighting_exponent_ratio = 1
end_intercept_weighting_exponent_ratio = 0
rsqr_weighting_exponent_ratio = 1
dispersion_weighting_exponent_ratio = 3

slope_weighting_exponent_predicted_actual = 0
intercept_weighting_exponent_predicted_actual = 0
rsqr_weighting_exponent_predicted_actual = 0
dispersion_weighting_exponent_predicted_actual = 0

analytical_parametrs = str(intercept_weighting_exponent_ratio)+str(slope_weighting_exponent_ratio)+str(end_intercept_weighting_exponent_ratio)+str(rsqr_weighting_exponent_ratio)+str(dispersion_weighting_exponent_ratio)
print(analytical_parametrs)

11013


#### Model Configs

In [9]:
'''Model parameters: in order to understand, consult the original documentation (case_version_cat Tab) '''
n_epochs = 100
batch = 64
correction_n_epochs =100
correction_batch=64

'''These are the exponent used to define the number of nodes for each layer'''
twoexp_nodes_number_layer_1 = 7
twoexp_nodes_number_layer_2 = 10
twoexp_nodes_number_layer_3 = 7
twoexp_nodes_number_layer_4 = 6
twoexp_nodes_number_layer_5 = 0

lr=0.0005 #learning rate
correction_lr=0.0005

In [10]:
'''Other variables to be used during training'''

max_iterations =1#maximum number of iterations for the while loop (we will ee later in the code)
precision = 0.00000000001 #this precision is related to the quality of the compound_run_term value we want to obtain (that is representative of the quality of the model)
attenuation_factor = 0.75 #it us used in the computation of the attenuated_padding_value (see custom_loss_function)

#### DATA

In [11]:
''' Conection to PostgreSQL '''
# The credentials to conect to the database
hostname = 'database-1.ctzm0hf7fhri.eu-central-1.rds.amazonaws.com'
database = 'dyDATA_new'
username = 'postgres'
pwd = 'Proc2023awsrdspostgresql'
port_id = 5432
conn = None

#this helps to retreive the data for a particular asset from the database
asset_script="SELECT * FROM "+'\"'+"ASSET_"+case+'\"'+".features_targets_input_view WHERE features_targets_input_view."+'\"'+"cleaned_raw_features_environment_PK"+'\"'+ "= 4"
asset_script

'SELECT * FROM "ASSET_TSLA".features_targets_input_view WHERE features_targets_input_view."cleaned_raw_features_environment_PK"= 4'

In [12]:
''' The active financial assets '''
# Here we select the active financial asset from the financial asset list table
try:
  with psycopg2.connect(
      host = hostname,
      dbname = database,
      user = username,
      password = pwd,
      port = port_id
  ) as conn:

    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
        print('You are connect to the Database:',database)
        select_script = asset_script # do not forget to set to asset_script when data has been uploaded
        cur.execute(select_script)
        data = cur.fetchall()
        cols = []
        # loop to create the dataframe that contains the active financial assets
        for rec in cur.description:
            cols.append(rec[0])
        dohlcav_mpnxp_data= pd.DataFrame(data = data, columns = cols)
        print('Your SQL has executed successfully')

except Exception as error:
  print(error)

finally:
  if conn is not None:
     conn.close()

You are connect to the Database: dyDATA_new
Your SQL has executed successfully


In [13]:
if case=='TSLA':
  dohlcav_mpnxp_data = dohlcav_mpnxp_data.loc[dohlcav_mpnxp_data['cleaned_raw_features_DCP_date_current_period'] >= '2020-01-01'].reset_index(drop=True)

if case == 'EURUSD=X':
  dohlcav_mpnxp_data['cleaned_raw_features_VTCP_volume_of_transactions_current_period'] = 1

dohlcav_mpnxp_data.head()

,cleaned_raw_features_id,cleaned_raw_features_DCP_date_current_period,calculated_features_DNCP,cleaned_raw_features_OPCP_open_price_current_period,cleaned_raw_features_HPCP_high_price_current_period,cleaned_raw_features_LPCP_low_price_current_period,cleaned_raw_features_CPCP_close_price_current_period,cleaned_raw_features_ACPCP_adjusted_close_price_current_period,cleaned_raw_features_VTCP_volume_of_transactions_current_period,cleaned_raw_features_environment_PK,...,calculated_targets_MPN7P,calculated_targets_HPN7P,calculated_targets_LPN7P,calculated_targets_MPN10P,calculated_targets_HPN10P,calculated_targets_LPN10P,calculated_targets_MPN20P,calculated_targets_HPN20P,calculated_targets_LPN20P,calculated_targets_environment_PK
0,2330,2020-01-02,43832.0,28.300000,28.713333,28.114000,28.684000,28.684000,142981500.0,4,...,31.580000,35.042,29.128000,32.415584,36.49400,29.128000,35.272333,43.533333,29.128000,4
1,2331,2020-01-03,43833.0,29.366667,30.266666,29.128000,29.534000,29.534000,266677500.0,4,...,32.119330,36.494,29.333332,32.908334,36.49400,29.333332,35.907750,52.409332,29.333332,4
2,2332,2020-01-06,43836.0,29.364668,30.104000,29.333332,30.102667,30.102667,151995010.0,4,...,32.809334,36.494,30.224000,33.554250,36.57200,30.224000,36.487000,64.599335,30.224000,4
3,2333,2020-01-07,43837.0,30.760000,31.442000,30.224000,31.270666,31.270666,268231500.0,4,...,33.232666,36.494,31.215334,34.132999,39.63333,31.215334,37.120666,64.599335,31.215334,4
4,2334,2020-01-08,43838.0,31.580000,33.232666,31.215334,32.809334,32.809334,467164500.0,4,...,33.933166,36.494,31.524668,34.778666,39.63333,31.524668,37.642000,64.599335,31.524668,4


In [14]:
filtered_columns_1=list(dohlcav_mpnxp_data.columns[:9])#to filter out the dates columns and features columns
filtered_columns_2=[x for x in dohlcav_mpnxp_data.columns if  targets[0][3:] in x ]#feature out the main target columns

#special condition for filtering if the main target is MPN1P
if model_case_version_main_target_code=='1':
  temp=filtered_columns_2[0]
  temp_2=filtered_columns_2[1]
  filtered_columns_2[0]=filtered_columns_2[2]
  filtered_columns_2[1]=temp
  filtered_columns_2[2]=temp_2


#to add the last two constant columns to the table
filtered_columns_3=['calculated_targets_HPN1P','calculated_targets_LPN1P']

filtered_columns=filtered_columns_1+filtered_columns_2+filtered_columns_3

print(filtered_columns)

dohlcav_mpnxp_data=dohlcav_mpnxp_data[filtered_columns]

dohlcav_mpnxp_data.head()

['cleaned_raw_features_id', 'cleaned_raw_features_DCP_date_current_period', 'calculated_features_DNCP', 'cleaned_raw_features_OPCP_open_price_current_period', 'cleaned_raw_features_HPCP_high_price_current_period', 'cleaned_raw_features_LPCP_low_price_current_period', 'cleaned_raw_features_CPCP_close_price_current_period', 'cleaned_raw_features_ACPCP_adjusted_close_price_current_period', 'cleaned_raw_features_VTCP_volume_of_transactions_current_period', 'calculated_targets_MPN5P', 'calculated_targets_HPN5P', 'calculated_targets_LPN5P', 'calculated_targets_HPN1P', 'calculated_targets_LPN1P']


,cleaned_raw_features_id,cleaned_raw_features_DCP_date_current_period,calculated_features_DNCP,cleaned_raw_features_OPCP_open_price_current_period,cleaned_raw_features_HPCP_high_price_current_period,cleaned_raw_features_LPCP_low_price_current_period,cleaned_raw_features_CPCP_close_price_current_period,cleaned_raw_features_ACPCP_adjusted_close_price_current_period,cleaned_raw_features_VTCP_volume_of_transactions_current_period,calculated_targets_MPN5P,calculated_targets_HPN5P,calculated_targets_LPN5P,calculated_targets_HPN1P,calculated_targets_LPN1P
0,2330,2020-01-02,43832.0,28.300000,28.713333,28.114000,28.684000,28.684000,142981500.0,30.924166,33.253334,29.128000,30.266666,29.128000
1,2331,2020-01-03,43833.0,29.366667,30.266666,29.128000,29.534000,29.534000,266677500.0,31.580000,33.253334,29.333332,30.104000,29.333332
2,2332,2020-01-06,43836.0,29.364668,30.104000,29.333332,30.102667,30.102667,151995010.0,32.119330,35.042000,30.224000,31.442000,30.224000
3,2333,2020-01-07,43837.0,30.760000,31.442000,30.224000,31.270666,31.270666,268231500.0,32.809334,36.494000,31.215334,33.232666,31.215334
4,2334,2020-01-08,43838.0,31.580000,33.232666,31.215334,32.809334,32.809334,467164500.0,33.933166,36.494000,31.524668,33.253334,31.524668


In [15]:
if model_case_version_main_target_code=='1':
  dohlcav_mpnxp_data.columns=["ID","DCP_date_current_period","DNCP_day_number_current_period","OPCP_open_price_current_period","HPCP_high_price_current_period","LPCP_low_price_current_period"
,"CPCP_close_price_current_period","ACPCP_adjusted_close_price_current_period","VTCP_volume_of_transactions_current_period","MPN"+model_case_version_main_target_code+"P_median_price_next_"+model_case_version_main_target_code+"_periods","HPN"+model_case_version_main_target_code+"P_highest_price_next_"+model_case_version_main_target_code+"_periods","LPN"+model_case_version_main_target_code+"P_lowest_price_next_"+model_case_version_main_target_code+"_periods","HPN1P_high_price_next_1_period",
"LPN1P_low_price_next_1_period"
]
else:
  dohlcav_mpnxp_data = dohlcav_mpnxp_data.rename(columns={"cleaned_raw_features_id":"ID",
                                "cleaned_raw_features_DCP_date_current_period": "DCP_date_current_period",
                                "calculated_features_DNCP":"DNCP_day_number_current_period",
                                "cleaned_raw_features_OPCP_open_price_current_period":"OPCP_open_price_current_period",
                                "cleaned_raw_features_HPCP_high_price_current_period":"HPCP_high_price_current_period",
                                "cleaned_raw_features_LPCP_low_price_current_period":"LPCP_low_price_current_period",
                                "cleaned_raw_features_CPCP_close_price_current_period": "CPCP_close_price_current_period",
                                "cleaned_raw_features_ACPCP_adjusted_close_price_current_period":"ACPCP_adjusted_close_price_current_period",
                                "cleaned_raw_features_VTCP_volume_of_transactions_current_period":"VTCP_volume_of_transactions_current_period",
                                filtered_columns_2[0]:"MPN"+model_case_version_main_target_code+"P_median_price_next_"+model_case_version_main_target_code+"_periods",
                                filtered_columns_2[1]:"HPN"+model_case_version_main_target_code+"P_highest_price_next_"+model_case_version_main_target_code+"_periods",
                                filtered_columns_2[2]:"LPN"+model_case_version_main_target_code+"P_lowest_price_next_"+model_case_version_main_target_code+"_periods",
                                filtered_columns_3[0]:"HPN1P_high_price_next_1_period",
                                filtered_columns_3[1]:"LPN1P_low_price_next_1_period",
                                })
  
dohlcav_mpnxp_data.head()

,ID,DCP_date_current_period,DNCP_day_number_current_period,OPCP_open_price_current_period,HPCP_high_price_current_period,LPCP_low_price_current_period,CPCP_close_price_current_period,ACPCP_adjusted_close_price_current_period,VTCP_volume_of_transactions_current_period,MPN5P_median_price_next_5_periods,HPN5P_highest_price_next_5_periods,LPN5P_lowest_price_next_5_periods,HPN1P_high_price_next_1_period,LPN1P_low_price_next_1_period
0,2330,2020-01-02,43832.0,28.300000,28.713333,28.114000,28.684000,28.684000,142981500.0,30.924166,33.253334,29.128000,30.266666,29.128000
1,2331,2020-01-03,43833.0,29.366667,30.266666,29.128000,29.534000,29.534000,266677500.0,31.580000,33.253334,29.333332,30.104000,29.333332
2,2332,2020-01-06,43836.0,29.364668,30.104000,29.333332,30.102667,30.102667,151995010.0,32.119330,35.042000,30.224000,31.442000,30.224000
3,2333,2020-01-07,43837.0,30.760000,31.442000,30.224000,31.270666,31.270666,268231500.0,32.809334,36.494000,31.215334,33.232666,31.215334
4,2334,2020-01-08,43838.0,31.580000,33.232666,31.215334,32.809334,32.809334,467164500.0,33.933166,36.494000,31.524668,33.253334,31.524668


In [16]:
#this variable defines the path to save the results of the LMB
model_path=case + '_DECEMBER_FIKRI'

print(model_path)
#to check the existence of the specified path so as to be sure of saving results in the right path
if os.path.exists(model_path):
  print("YES")
else:
  print("NO")

TSLA_DECEMBER_FIKRI
NO


In [17]:
# dohlcav_mpnxp_data = dohlcav_mpnxp_data.replace(',','', regex=True) #remove the ',' otherwise it's impossible to deal with numbers in the dataset
dohlcav_mpnxp_data.tail(int(model_case_version_main_target_code)+1) # to visualize likely columns with NaN values in the dataset


,ID,DCP_date_current_period,DNCP_day_number_current_period,OPCP_open_price_current_period,HPCP_high_price_current_period,LPCP_low_price_current_period,CPCP_close_price_current_period,ACPCP_adjusted_close_price_current_period,VTCP_volume_of_transactions_current_period,MPN5P_median_price_next_5_periods,HPN5P_highest_price_next_5_periods,LPN5P_lowest_price_next_5_periods,HPN1P_high_price_next_1_period,LPN1P_low_price_next_1_period
1009,3339,2024-01-05,45296.0,236.86,240.1196,234.9001,237.49,237.49,92084160.0,233.94,241.24,217.1501,241.2400,235.3000
1010,3340,2024-01-08,45299.0,236.14,241.2400,235.3000,240.45,240.45,84705016.0,233.94,241.24,217.1501,238.9646,232.0400
1011,3341,2024-01-09,45300.0,238.11,238.9646,232.0400,234.96,234.96,96267000.0,233.94,241.24,217.1501,235.5000,231.2900
1012,3342,2024-01-10,45301.0,235.10,235.5000,231.2900,233.94,233.94,91483000.0,233.94,241.24,217.1501,230.9150,225.3700
1013,3343,2024-01-11,45302.0,230.57,230.9150,225.3700,227.22,227.22,104632210.0,233.94,241.24,217.1501,225.3400,217.1501
1014,3344,2024-01-12,45303.0,220.08,225.3400,217.1501,218.89,218.89,122687424.0,233.94,241.24,217.1501,225.3400,217.1501


In [18]:
''' This comprises the list of target in our datasets'''
targets_list=["MPN"+model_case_version_main_target_code+"P_median_price_next_"+model_case_version_main_target_code+"_periods",
                        "HPN"+model_case_version_main_target_code+"P_highest_price_next_"+model_case_version_main_target_code+"_periods",
                        "LPN"+model_case_version_main_target_code+"P_lowest_price_next_"+model_case_version_main_target_code+"_periods",
                        'HPN1P_high_price_next_1_period','LPN1P_low_price_next_1_period']

targets_list

['MPN5P_median_price_next_5_periods',
 'HPN5P_highest_price_next_5_periods',
 'LPN5P_lowest_price_next_5_periods',
 'HPN1P_high_price_next_1_period',
 'LPN1P_low_price_next_1_period']

In [19]:
for i in targets_list:
  if targets[0] in i:
    main_target_column=i

main_target_column

'MPN5P_median_price_next_5_periods'

In [20]:
''' What is stop target?'''
stop_target=dohlcav_mpnxp_data.index[-1]+1

In [21]:
''' Here what we do is to separate data in two parts: the first goes from the beginnig of 2020 to the end of 2021 and
it is used for training the model; the second goes from the beginning of 2022 until the end
(it dependes on the last update we did for the dataset) and it is used for testing the model.
To do that, we siply compute the size of the training set and then we use this value (later in the code) to split the dataset '''

dohlcav_mpnxp_data['DCP_date_current_period'] = pd.to_datetime(dohlcav_mpnxp_data['DCP_date_current_period']) #date values in the dataset are converted

dohlcav_mpnxp_data.head()

,ID,DCP_date_current_period,DNCP_day_number_current_period,OPCP_open_price_current_period,HPCP_high_price_current_period,LPCP_low_price_current_period,CPCP_close_price_current_period,ACPCP_adjusted_close_price_current_period,VTCP_volume_of_transactions_current_period,MPN5P_median_price_next_5_periods,HPN5P_highest_price_next_5_periods,LPN5P_lowest_price_next_5_periods,HPN1P_high_price_next_1_period,LPN1P_low_price_next_1_period
0,2330,2020-01-02,43832.0,28.300000,28.713333,28.114000,28.684000,28.684000,142981500.0,30.924166,33.253334,29.128000,30.266666,29.128000
1,2331,2020-01-03,43833.0,29.366667,30.266666,29.128000,29.534000,29.534000,266677500.0,31.580000,33.253334,29.333332,30.104000,29.333332
2,2332,2020-01-06,43836.0,29.364668,30.104000,29.333332,30.102667,30.102667,151995010.0,32.119330,35.042000,30.224000,31.442000,30.224000
3,2333,2020-01-07,43837.0,30.760000,31.442000,30.224000,31.270666,31.270666,268231500.0,32.809334,36.494000,31.215334,33.232666,31.215334
4,2334,2020-01-08,43838.0,31.580000,33.232666,31.215334,32.809334,32.809334,467164500.0,33.933166,36.494000,31.524668,33.253334,31.524668


In [22]:
start_date = pd.Timestamp(str(dohlcav_mpnxp_data['DCP_date_current_period'].iloc[0])) #start date of the training set

train_end_date = pd.Timestamp('2022-12-30')#this specifies the training end date

period = str(start_date.date()) + '_' + str(train_end_date.date())

train_end_idx = dohlcav_mpnxp_data.index[dohlcav_mpnxp_data['DCP_date_current_period']==train_end_date].values[0]
train_end_idx = train_end_idx-int(model_case_version_main_target_code)

train_end_date=dohlcav_mpnxp_data.loc[train_end_idx,'DCP_date_current_period']#we move the training date in such a way that we avoid the tail values
prediction_end_date=pd.Timestamp(str(dohlcav_mpnxp_data['DCP_date_current_period'].iloc[-1]))

filename = 'proceedit '+today+' SPP-'+case+'_'+period+'_MPN'+model_case_version_main_target_code+'P_LSTM-15710760-B'+str(batch)+'E'+str(n_epochs)+'L'+str(lr)+'T'+model_case_version_time_steps+ \
            '-DOHLCAV-FTEKR_LVSP_LSTM-15710760-B'+str(batch)+'E'+str(n_epochs)+'L'+str(lr)+'T'+model_case_version_time_steps+'-PR_'+analytical_parametrs+'L0.75_AH'

print(filename)

proceedit 20240116 SPP-TSLA_2020-01-02_2022-12-30_MPN5P_LSTM-15710760-B64E100L0.0005T10-DOHLCAV-FTEKR_LVSP_LSTM-15710760-B64E100L0.0005T10-PR_11013L0.75_AH


In [23]:
'''This is the mask of booleans that will be used to filter data and take just what we need (data from the beginning until the end of 2021)'''

train_mask = (dohlcav_mpnxp_data['DCP_date_current_period'] <= train_end_date)#to select data for training
prediction_mask = (dohlcav_mpnxp_data['DCP_date_current_period'] <= prediction_end_date)#to select data for prediction

In [24]:
training_size = dohlcav_mpnxp_data.loc[train_mask].shape[0] #the mask is applied and from the correspondent dataframe we take just the shape[0] (the size\the number of rows)
prediction_size = dohlcav_mpnxp_data.loc[prediction_mask].shape[0]
print('Training size: ', training_size)
print('Prediction size: ', prediction_size)

Training size:  751
Prediction size:  1015


In [25]:
'''These will be used in the predictions output file (in order to know from which point starting to paste the results).'''
dates = dohlcav_mpnxp_data.iloc[int(model_case_version_time_steps)-1:,1].apply(lambda x: x.date().strftime('%Y-%m-%d')).reset_index(drop=True)
dates

0       2020-01-15
1       2020-01-16
2       2020-01-17
3       2020-01-21
4       2020-01-22
           ...    
1001    2024-01-08
1002    2024-01-09
1003    2024-01-10
1004    2024-01-11
1005    2024-01-12
Name: DCP_date_current_period, Length: 1006, dtype: object

In [26]:
'''Building the dataframe with just the necessary columns (removing 'id', 'uuid' and 'ACPCP_adjusted_close_price_current_period' column)'''

#pay attention here because everytime targets change, also the name of the columns change
df = dohlcav_mpnxp_data.drop(["ID","DCP_date_current_period"], axis=1)

'''Renaming columns to have a more compact and a better reading of the df'''
#pay attention here because everytime targets change, also the name of the columns change
df = df.rename(columns={"DNCP_day_number_current_period": "DNCP",
                        "OPCP_open_price_current_period":"OPCP",
                        "HPCP_high_price_current_period":"HPCP",
                        "LPCP_low_price_current_period":"LPCP",
                        "CPCP_close_price_current_period":"CPCP",
                        "ACPCP_adjusted_close_price_current_period": "ACPCP",
                        "VTCP_volume_of_transactions_current_period":"VTCP",
                        "MPN"+model_case_version_main_target_code+"P_median_price_next_"+model_case_version_main_target_code+"_periods":"MPN"+model_case_version_main_target_code+"P",
                        "HPN"+model_case_version_main_target_code+"P_highest_price_next_"+model_case_version_main_target_code+"_periods":"HPN"+model_case_version_main_target_code+"P",
                        "LPN"+model_case_version_main_target_code+"P_lowest_price_next_"+model_case_version_main_target_code+"_periods":"LPN"+model_case_version_main_target_code+"P",
                        'HPN1P_high_price_next_1_period':'hpn1p',
                        'LPN1P_low_price_next_1_period':'lpn1p'})

#df.columns = ['DNCP', 'OPCP', 'HPCP', 'LPCP', 'CPCP', 'ACPCP', 'VTCP', 'MPN1P', 'HPN1P', 'LPN1P', 'HPN1P', 'LPN1P']

df.head()

,DNCP,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,MPN5P,HPN5P,LPN5P,hpn1p,lpn1p
0,43832.0,28.300000,28.713333,28.114000,28.684000,28.684000,142981500.0,30.924166,33.253334,29.128000,30.266666,29.128000
1,43833.0,29.366667,30.266666,29.128000,29.534000,29.534000,266677500.0,31.580000,33.253334,29.333332,30.104000,29.333332
2,43836.0,29.364668,30.104000,29.333332,30.102667,30.102667,151995010.0,32.119330,35.042000,30.224000,31.442000,30.224000
3,43837.0,30.760000,31.442000,30.224000,31.270666,31.270666,268231500.0,32.809334,36.494000,31.215334,33.232666,31.215334
4,43838.0,31.580000,33.232666,31.215334,32.809334,32.809334,467164500.0,33.933166,36.494000,31.524668,33.253334,31.524668


In [27]:
pd.set_option('display.max_columns',None)


base_target_code = 'MPN' + model_case_version_main_target_code + 'P'
base_target_column_index = df.columns.get_loc(base_target_code)

def new_target_column(target_code , shift_back_period):
  prev_target = df[target_code]
  new_target = prev_target[:-shift_back_period]
  first_dates_handling = [0] * shift_back_period
  new_target=np.concatenate((first_dates_handling,new_target))
  return new_target

#Adding multiple targets
new_target_index = base_target_column_index
for i in range(int(model_case_version_main_target_code)):
  new_target_code = 'MPN-' + str(i+1) + 'P'
  df.insert(new_target_index+1,new_target_code,new_target_column(base_target_code,i+1))
  new_target_index = new_target_index + 1
  targets.append(new_target_code)

# '''Ratio Transformation for features'''
df.insert(7,'OPCP_Ratio',df['OPCP']/df['CPCP'])
df.insert(8,'HPCP_Ratio',df['HPCP']/df['CPCP'])
df.insert(9,'LPCP_Ratio',df['LPCP']/df['CPCP'])
df.insert(10,'ACPCP_Ratio',df['ACPCP']/df['CPCP'])
df.insert(df.columns.get_loc('MPN'+model_case_version_main_target_code+'P') ,'MPN'+ model_case_version_main_target_code +'P_Ratio',df['MPN'+ model_case_version_main_target_code +'P'].shift(5)/df['CPCP'].shift(5))
df.insert(df.columns.get_loc('HPN'+model_case_version_main_target_code+'P'),'HPN'+ model_case_version_main_target_code +'P_Ratio',df['HPN'+ model_case_version_main_target_code +'P']/df['CPCP'])
df.insert(df.columns.get_loc('LPN'+model_case_version_main_target_code+'P'),'LPN'+ model_case_version_main_target_code +'P_Ratio',df['LPN'+ model_case_version_main_target_code +'P']/df['CPCP'])
df.insert(df.columns.get_loc('hpn1p')+1,'hpn1p_Ratio',df['hpn1p']/df['CPCP'])
df.insert(df.columns.get_loc('lpn1p')+1,'lpn1p_Ratio',df['lpn1p']/df['CPCP'])

df = df.iloc[int(model_case_version_main_target_code):]

df.head()

,DNCP,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,OPCP_Ratio,HPCP_Ratio,LPCP_Ratio,ACPCP_Ratio,MPN5P_Ratio,MPN5P,MPN-1P,MPN-2P,MPN-3P,MPN-4P,MPN-5P,HPN5P_Ratio,HPN5P,LPN5P_Ratio,LPN5P,hpn1p,hpn1p_Ratio,lpn1p,lpn1p_Ratio
5,43839.0,33.140000,33.253334,31.524668,32.089333,32.089333,426606020.0,1.032742,1.036274,0.982403,1.0,1.078098,34.297333,33.933166,32.809334,32.119330,31.580000,30.924166,1.137263,36.49400,0.984128,31.580000,32.329334,1.007479,31.580000,0.984128
6,43840.0,32.119330,32.329334,31.580000,31.876667,31.876667,194392500.0,1.007613,1.014201,0.990693,1.0,1.069276,34.378000,34.297333,33.933166,32.809334,32.119330,31.580000,1.144850,36.49400,1.028966,32.800000,35.042000,1.099299,32.800000,1.028966
7,43843.0,32.900000,35.042000,32.800000,34.990665,34.990665,397764000.0,0.940251,1.001467,0.937393,1.0,1.066993,34.993332,34.378000,34.297333,33.933166,32.809334,32.119330,1.045193,36.57200,0.937717,32.811333,36.494000,1.042964,34.993332,1.000076
8,43844.0,36.284000,36.494000,34.993332,35.861332,35.861332,434943000.0,1.011786,1.017642,0.975796,1.0,1.049205,35.048167,34.993332,34.378000,34.297333,33.933166,32.809334,1.105183,39.63333,0.914950,32.811333,35.856000,0.999851,34.452667,0.960719
9,43845.0,35.317333,35.856000,34.452667,34.566666,34.566666,260532000.0,1.021716,1.037300,0.996702,1.0,1.034253,35.907333,35.048167,34.993332,34.378000,34.297333,33.933166,1.146577,39.63333,0.949219,32.811333,34.297333,0.992208,32.811333,0.949219


In [28]:
for target in targets:
  avg_prices_list.append(df[target].astype(float).mean())

avg_prices_list

[210.89792031089107,
 210.69989374282179,
 210.50075446955447,
 210.30093202400988,
 210.10057558836635,
 209.89956981262375]

In [29]:
'''?????????????????'''

main_target_code_integer = int(model_case_version_main_target_code)

#this is calculate the period day number to be used in the training section
dncp_train = dohlcav_mpnxp_data[train_mask]['DNCP_day_number_current_period'].replace(',', '', regex=True)[int(model_case_version_time_steps)-1+main_target_code_integer:]
dncp_train = dncp_train.astype(int).to_numpy()

span_dncp_train=dncp_train[-1] - dncp_train[0] + 1

positions_day_number_train= dncp_train-dncp_train[0]+1
positions_day_number_train

array([   1,    2,    5,    6,    7,    8,    9,   12,   13,   14,   15,
         16,   19,   20,   21,   22,   23,   27,   28,   29,   30,   33,
         34,   35,   36,   37,   40,   41,   42,   43,   44,   47,   48,
         49,   50,   51,   54,   55,   56,   57,   58,   61,   62,   63,
         64,   65,   68,   69,   70,   71,   72,   75,   76,   77,   78,
         82,   83,   84,   85,   86,   89,   90,   91,   92,   93,   96,
         97,   98,   99,  100,  103,  104,  105,  106,  107,  110,  111,
        112,  113,  114,  117,  118,  119,  120,  121,  125,  126,  127,
        128,  131,  132,  133,  134,  135,  138,  139,  140,  141,  142,
        145,  146,  147,  148,  149,  152,  153,  154,  155,  156,  159,
        160,  161,  162,  166,  167,  168,  169,  170,  173,  174,  175,
        176,  177,  180,  181,  182,  183,  184,  187,  188,  189,  190,
        191,  194,  195,  196,  197,  198,  201,  202,  203,  204,  205,
        208,  209,  210,  211,  212,  215,  216,  2

In [30]:
int(model_case_version_time_steps) - 1 + main_target_code_integer

14

In [31]:
#this is calculate the period day number to be used in the prediction section
dncp = dohlcav_mpnxp_data['DNCP_day_number_current_period'].replace(',','', regex=True)[int(model_case_version_time_steps)-1+main_target_code_integer:]
dncp = dncp.astype(int).to_numpy()
span_dncp=dncp[-1] - dncp[0] + 1
positions_day_number = dncp-dncp[0]+1
positions_day_number

array([   1,    2,    5, ..., 1449, 1450, 1451])

In [32]:
len(positions_day_number)

1001

In [33]:
'''The padding point value is calculated for computing the value at the end of the trend line. We'll see better during the training and the application of vertical padding '''
padding_point = positions_day_number[0]
padding_point

1

### Pretreatment

In [34]:
#seperate training dataset and apply pre-treament only on the training dataset
train_dataset=df.iloc[main_target_code_integer:train_end_idx+1,:]

train_dataset.head()

,DNCP,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,OPCP_Ratio,HPCP_Ratio,LPCP_Ratio,ACPCP_Ratio,MPN5P_Ratio,MPN5P,MPN-1P,MPN-2P,MPN-3P,MPN-4P,MPN-5P,HPN5P_Ratio,HPN5P,LPN5P_Ratio,LPN5P,hpn1p,hpn1p_Ratio,lpn1p,lpn1p_Ratio
10,43846.0,32.916668,34.297333,32.811333,34.232666,34.232666,326050500.0,0.961557,1.001889,0.958480,1.0,1.068808,37.273335,35.907333,35.048167,34.993332,34.378000,34.297333,1.157763,39.63333,0.979883,33.544000,34.37800,1.004245,33.544000,0.979883
11,43847.0,33.840668,34.378000,33.544000,34.033333,34.033333,204436500.0,0.994339,1.010127,0.985622,1.0,1.078469,37.616670,37.273335,35.907333,35.048167,34.993332,34.378000,1.164544,39.63333,1.035083,35.227333,36.57200,1.074594,35.227333,1.035083
12,43851.0,35.350000,36.572000,35.227333,36.480000,36.480000,267052500.0,0.969024,1.002522,0.965662,1.0,1.000076,37.793330,37.616670,37.273335,35.907333,35.048167,34.993332,1.086440,39.63333,0.985526,35.952000,39.63333,1.086440,37.273335,1.021747
13,43852.0,38.126000,39.633330,37.273335,37.970670,37.970670,470535000.0,1.004091,1.043788,0.981635,1.0,0.977325,37.828667,37.793330,37.616670,37.273335,35.907333,35.048167,1.035536,39.32000,0.946836,35.952000,38.80000,1.021841,37.040000,0.975490
14,43853.0,37.616670,38.800000,37.040000,38.146667,38.146667,294765000.0,0.986106,1.017127,0.970989,1.0,1.038785,37.899334,37.828667,37.793330,37.616670,37.273335,35.907333,1.137504,43.39200,0.942468,35.952000,38.25733,1.002901,36.950670,0.968647


In [35]:
# Pretreatment Train

train_df_array=train_dataset.to_numpy(dtype='float64')

#application of logarithmic detrending
if not (train_df_array < 0).any():
    print('logarithmic detrending applied')
    train_df_array = np.log(train_df_array)


print('--------- Logatihmic Transform ---------')
#print(pd.DataFrame(train_df_array, columns=train_dataset.columns, index=train_dataset.index))

kalman_train=True#not applying kalman Filter


'''Preprocessing consists, in this case, to transform the dataset through 3 filters: Kalman, E_logrithmic and RobustScaler (in this order)'''
if kalman_train: #necessary to deal with filter results
  '''Application of the Kalman filter (rounding data)'''
  kalman_smoother=tsmoothie.KalmanSmoother(component='level_trend',  component_noise={'level':0.1, 'trend':0.1}) #values for Kalman filters parameters
                                                                                                                  #are taken from an example in the original code
                                                                                                                  #of the library
  temp_df=pd.DataFrame(train_df_array,columns=train_dataset.columns)
  for i in range(len(temp_df.columns)): #this is the more convenient way I've found to apply the filter
    kalman_smoother.smooth(temp_df[temp_df.columns[i]])
    train_df_array[:,i] = kalman_smoother.smooth_data

print('--------- Kalman Smooth ---------')
#print(pd.DataFrame(train_df_array, columns=train_dataset.columns, index=train_dataset.index))
pd.DataFrame(train_df_array, columns=train_dataset.columns, index=train_dataset.index)


#fitting the robust scaler onn both the features and target column separately for the training dataset
train_robust_scaler_features= RobustScaler().fit(train_df_array[:,:12])
train_robust_scaler_target=RobustScaler().fit(train_df_array[:,12:12+int(model_case_version_main_target_code)+1])

#performing transform on the features
train_df_features=train_robust_scaler_features.transform(train_df_array[:,:12])
#performing transform on the target
train_df_target=train_robust_scaler_target.transform(train_df_array[:,12:12+int(model_case_version_main_target_code)+1])


logarithmic detrending applied
--------- Logatihmic Transform ---------
--------- Kalman Smooth ---------


In [36]:
df_array=df.to_numpy(dtype='float64')


# df_array = df_array[5:]
#applying logarithmic detrending to the prediction/full dataset
if not (df_array < 0).any():
    df_array = np.log(df_array)

kalman_predict=True
if kalman_predict:
  '''Preprocessing consists, in this case, to transform the dataset through 3 filters: Kalman, E_logrithmic and RobustScaler (in this order)'''

  '''Application of the Kalman filter (rounding data)'''
  kalman_smoother=tsmoothie.KalmanSmoother(component='level_trend',  component_noise={'level':0.1, 'trend':0.1}) #values for Kalman filters parameters
                                                                                                                  #are taken from an example in the original code
                                                                                                                  #of the library
  temp_df=pd.DataFrame(df_array,columns=df.columns)
  for i in range(len(df.columns)): #this is the more convenient way I've found to apply the filter
    kalman_smoother.smooth(temp_df[temp_df.columns[i]])
    df_array[:,i] = kalman_smoother.smooth_data

#robust scaler transform on the features
prediction_df_features=train_robust_scaler_features.transform(df_array[:,:12])
#robust scaler transform on the target
prediction_df_targets=train_robust_scaler_target.transform(df_array[:,12:12+int(model_case_version_main_target_code)+1])

In [37]:
'''Creating the input blocks for the models. The timestep value must be changed according to the targets and dataset we are working with'''
X_train, y_train =building_data_sequences(train_df_features,train_df_target,timesteps=int(model_case_version_time_steps))
X_predict,y_predict=building_data_sequences(prediction_df_features,prediction_df_targets,timesteps=int(model_case_version_time_steps))#see Functions section above;
#X_test, y_test = building_data_sequences(test_data, timesteps=int(model_case_version_time_steps))

print(X_train.shape)
print(X_predict.shape)

(737, 10, 12)
(1001, 10, 12)


### Train Configurations

In [38]:
'''In this section, we're going to define some variables that will be useful during the training and testing of the model'''

input_shape=((X_train).shape[1],(X_train).shape[2])
print("Input shape obtained is:",input_shape)

'''We need actual values (true values) to compute some analytical parameters during the training'''
df_actual = df.iloc[int(model_case_version_time_steps)-1:,:].reset_index(drop=True)
print(df_actual)

actuals_cols = [] #simply put the targets columns into a list (in order to be iterated during training)
for target in targets:
  actuals_cols.append(np.array(df_actual[target].astype(float)))

actuals_cols

Input shape obtained is: (10, 12)
         DNCP        OPCP        HPCP        LPCP        CPCP       ACPCP  \
0     43853.0   37.616670   38.800000   37.040000   38.146667   38.146667   
1     43854.0   38.042000   38.257330   36.950670   37.654667   37.654667   
2     43857.0   36.132668   37.629333   35.952000   37.201332   37.201332   
3     43858.0   37.899334   38.454000   37.205334   37.793330   37.793330   
4     43859.0   38.379333   39.320000   37.828667   38.732666   38.732666   
...       ...         ...         ...         ...         ...         ...   
996   45299.0  236.140000  241.240000  235.300000  240.450000  240.450000   
997   45300.0  238.110000  238.964600  232.040000  234.960000  234.960000   
998   45301.0  235.100000  235.500000  231.290000  233.940000  233.940000   
999   45302.0  230.570000  230.915000  225.370000  227.220000  227.220000   
1000  45303.0  220.080000  225.340000  217.150100  218.890000  218.890000   

             VTCP  OPCP_Ratio  HPCP_Ratio

[array([ 37.899334 ,  38.5651665,  42.168    , ..., 233.94     ,
        233.94     , 233.94     ]),
 array([ 37.828667 ,  37.899334 ,  38.5651665, ..., 233.94     ,
        233.94     , 233.94     ]),
 array([ 37.79333 ,  37.828667,  37.899334, ..., 233.94    , 233.94    ,
        233.94    ]),
 array([ 37.61667 ,  37.79333 ,  37.828667, ..., 233.94    , 233.94    ,
        233.94    ]),
 array([ 37.273335,  37.61667 ,  37.79333 , ..., 235.3     , 233.94    ,
        233.94    ]),
 array([ 35.90733325,  37.273335  ,  37.61667   , ..., 237.342425  ,
        235.3       , 233.94      ])]

In [39]:
'''??????????????'''
'''this indicates the index from which we start to replace the actual target with the predicted target to be used in the prediction section'''
stop_actual=stop_target-int(model_case_version_time_steps)+1-main_target_code_integer
stop_actual

1001

In [40]:
best_models_path = model_path + '/best_models/' #this path is used just to make everything more orderly

### Train

In [41]:
df_cpcp_train=df.iloc[(int(model_case_version_time_steps)-1):train_end_idx+1,:].reset_index(drop=True)
CPCP_cols_train=[]
CPCP_cols_train.append(np.array(df_cpcp_train['CPCP'].astype(float)))

In [42]:
best_prediction_run=0
best_correction_run=0

prediction_runs=100
correction_runs=1

past_corr_train_stat=True
past_corr_pred_stat=True

apply_correction_lstm=True
apply_vertical_swing_corr=True

In [43]:
'''This is the custom loss function we defined in order to apply a sort of padding during the training of the model. It consists in a normal mean squared error loss fnuction except for the application of
a multiplicative factor on the prediction of the model'''

def custom_loss_function(attenuated_padding_value):

  def padding_loss_function(y_true, y_pred):

    y_pred = tf.multiply(y_pred, attenuated_padding_value) #this is the multiplication between the predictions and the attenuated_padding_value

    squared_difference = tf.square(y_true - y_pred)

    return tf.reduce_mean(squared_difference, axis=-1) #mse

  return padding_loss_function

intercepts_and_slopes = {'intercepts': [],
                         'slopes':[],
                         'end_intercepts':[]
                         }

In [44]:
'''This is about a part that must be inserted in the analytical paramters file and which is referred to the weights that are used to compute the compound_run_term (see later in the code)'''

weights_df = pd.DataFrame([None, None, None, None, None, None, None, None, slope_weighting_exponent_ratio, intercept_weighting_exponent_ratio, end_intercept_weighting_exponent_ratio, rsqr_weighting_exponent_ratio, dispersion_weighting_exponent_ratio,
                           slope_weighting_exponent_predicted_actual, intercept_weighting_exponent_predicted_actual, rsqr_weighting_exponent_predicted_actual, dispersion_weighting_exponent_predicted_actual, None, None, None, None,None]).T

weights_df=weights_df.rename({0:'weights'})
weights_df=weights_df.rename(columns={0:'slope_predicted_calculated_target_ratio_versus_period',
                           1: 'intercept_predicted_calculated_target_ratio_versus_period',
                           2: 'end_intercept_predicted_calculated_target_ratio_versus_period',
                           3: 'rsqr_predicted_calculated_target_ratio_versus_period',
                           4: 'average_tld_predicted_calculated_target_ratio_versus_period',
                           5: 'slope_predicted_versus_calculated_target',
                           6: 'intercept_predicted_versus_calculated_target',
                           7: 'rsqr_predicted_versus_calculated_target',
                           8: 'average_tld_predicted_versus_calculated_target',
                           9: 'normalized_slope_predicted_calculated_target_ratio_versus_period',
                           10: 'normalized_intercept_predicted_calculated_target_ratio_versus_period',
                           11: 'normalized_end_intercept_predicted_calculated_target_ratio_versus_period',
                           12: 'normalized_rsqr_predicted_calculated_target_ratio_versus_period',
                           13: 'rescaled_normalized_average_tld_predicted_calculated_target_ratio_versus_period',
                           14: 'normalized_slope_predicted_versus_calculated_target',
                           15: 'rescaled_normalized_intercept_predicted_versus_calculated_target',
                           16: 'normalized_rsqr_predicted_versus_calculated_target',
                           17: 'rescaled_normalized_average_tld_predicted_versus_calculated_target',
                           18: 'compound_run_term',
                           19: 'best_run',
                           20: 'adjusted_compound_run_term',
                           21: 'padding_correction_factor',
                           22: 'padding_correction_factor_attenuation'
                           })

weights_df.columns

Index(['slope_predicted_calculated_target_ratio_versus_period',
       'intercept_predicted_calculated_target_ratio_versus_period',
       'end_intercept_predicted_calculated_target_ratio_versus_period',
       'rsqr_predicted_calculated_target_ratio_versus_period',
       'average_tld_predicted_calculated_target_ratio_versus_period',
       'slope_predicted_versus_calculated_target',
       'intercept_predicted_versus_calculated_target',
       'rsqr_predicted_versus_calculated_target',
       'average_tld_predicted_versus_calculated_target',
       'normalized_slope_predicted_calculated_target_ratio_versus_period',
       'normalized_intercept_predicted_calculated_target_ratio_versus_period',
       'normalized_end_intercept_predicted_calculated_target_ratio_versus_period',
       'normalized_rsqr_predicted_calculated_target_ratio_versus_period',
       'rescaled_normalized_average_tld_predicted_calculated_target_ratio_versus_period',
       'normalized_slope_predicted_versus_calcula

In [45]:
'''The creation of the new analytical parameters output file '''

new_weights_df = pd.DataFrame([slope_weighting_exponent_ratio, intercept_weighting_exponent_ratio,
                               end_intercept_weighting_exponent_ratio, rsqr_weighting_exponent_ratio,
                               dispersion_weighting_exponent_ratio, slope_weighting_exponent_predicted_actual,
                               intercept_weighting_exponent_predicted_actual, end_intercept_weighting_exponent_ratio,
                               rsqr_weighting_exponent_predicted_actual, dispersion_weighting_exponent_predicted_actual,
                               None, None, None, None]).T
new_weights_df = new_weights_df.rename({0:'weights'})
new_weights_df = new_weights_df.rename(columns={0:'normalized_trend_slope',
                               1: 'normalized_trend_intercept',
                               2: 'normalized_trend_end_intercept',
                               3: 'normalized_trend_rsqr',
                               4: 'rescaled_normalized_trend_dispersion',
                               5: 'trend_slope_weighting_exponent',
                               6: 'trend_intercept_weighting_exponent',
                               7: 'trend_end_intercept_weighting_exponent',
                               8: 'trend_rsqr_weighting_exponent',
                               9: 'trend_dispersion_weighting_exponent',
                               10: 'adjusted_compound_run_value',
                               11: 'number_of_runs_performed',
                               12: 'best_run_number',
                               13: 'best_model_.h5_file_link'
                               })

new_weights_df.columns

Index(['normalized_trend_slope', 'normalized_trend_intercept',
       'normalized_trend_end_intercept', 'normalized_trend_rsqr',
       'rescaled_normalized_trend_dispersion',
       'trend_slope_weighting_exponent', 'trend_intercept_weighting_exponent',
       'trend_end_intercept_weighting_exponent',
       'trend_rsqr_weighting_exponent', 'trend_dispersion_weighting_exponent',
       'adjusted_compound_run_value', 'number_of_runs_performed',
       'best_run_number', 'best_model_.h5_file_link'],
      dtype='object')

In [46]:
#this prepare the correction lstm table consisting of three columns:period_day_number, raw_predicted_targets,actual
def preprare_correction_lstm_table(period_day_number,raw_predicted_targets,volume,actual):
  correction_lstm_table=pd.DataFrame()
  correction_lstm_table['period_day_number']=period_day_number[int(model_case_version_main_target_code):]
  correction_lstm_table['raw_predicted_targets']=raw_predicted_targets[int(model_case_version_main_target_code):]
  new = (actual/raw_predicted_targets)
  print(len(new[:-int(model_case_version_main_target_code)]))
  print(new)
  correction_lstm_table['new_feature'] = new[:-int(model_case_version_main_target_code)]
  print(correction_lstm_table['new_feature'])
  correction_lstm_table['volume'] = volume[int(model_case_version_main_target_code):]
  correction_lstm_table['actual_raw_predited_ratio']=(actual/raw_predicted_targets)[int(model_case_version_main_target_code):]
  return correction_lstm_table

#this prepare the correction lstm table consisting of three columns:period_day_number, raw_predicted_targets,actual
def preprare_correction_lstm_table_predict(period_day_number,raw_predicted_targets,volume,actual,lstm_scaler_feat,lstm_scaler_target):
  correction_lstm_table=pd.DataFrame()
  correction_lstm_table['period_day_number']=period_day_number[int(model_case_version_main_target_code):]
  correction_lstm_table['raw_predicted_targets']=raw_predicted_targets[int(model_case_version_main_target_code):]
  new = (actual/raw_predicted_targets)
  correction_lstm_table['new_feature'] = new[:-int(model_case_version_main_target_code)]
  correction_lstm_table['volume'] = volume[int(model_case_version_main_target_code):]
  correction_lstm_table['actual_raw_predited_ratio']=(actual/raw_predicted_targets)[int(model_case_version_main_target_code):]
  correction_lstm_table=correction_lstm_table.to_numpy()
  train_df_lstm_features=lstm_scaler_feat.transform(correction_lstm_table[:,:2])
  train_df_lstm_target=lstm_scaler_target.transform(correction_lstm_table[:,2].reshape(-1,1))
  final_correction_lstm=np.concatenate((train_df_lstm_features,train_df_lstm_target),axis=1)
  return final_correction_lstm

#this prepare the correction lstm table consisting of three columns:period_day_number, raw_predicted_targets,actual
def preprare_correction_lstm_table_train(period_day_number, raw_predicted_targets,volume,actual):
  correction_lstm_table_temp=pd.DataFrame()
  correction_lstm_table_temp['period_day_number']=period_day_number[int(model_case_version_main_target_code):]
  correction_lstm_table_temp['raw_predicted_targets']=raw_predicted_targets[int(model_case_version_main_target_code):]
  new = (actual/raw_predicted_targets)
  # print(len(new[:-int(model_case_version_main_target_code)]))
  # print(new)
  # sys.exit()
  correction_lstm_table_temp['new_feature'] = new[:-int(model_case_version_main_target_code)]
  # print(correction_lstm_table['new_feature'])
  correction_lstm_table_temp['volume'] = volume[int(model_case_version_main_target_code):]
  correction_lstm_table_temp['actual_raw_predited_ratio']=(actual/raw_predicted_targets)[int(model_case_version_main_target_code):]
  # pd.set_option('display.max_rows', None)
  print(correction_lstm_table_temp)
  pd.reset_option('display.max_rows')
  correction_lstm_table=correction_lstm_table_temp.to_numpy()
  correction_lstm_table = correction_lstm_table[int(model_case_version_main_target_code):]
  robust_scaler_LSTM_features= RobustScaler().fit(correction_lstm_table[:,:4])
  robust_scaler_LSTM_target=RobustScaler().fit(correction_lstm_table[:,4].reshape(-1,1))
  train_df_lstm_features=robust_scaler_LSTM_features.transform(correction_lstm_table[:,:4])
  train_df_lstm_target=robust_scaler_LSTM_target.transform(correction_lstm_table[:,4].reshape(-1,1))
  final_correction_lstm=np.concatenate((train_df_lstm_features,train_df_lstm_target),axis=1)
  return final_correction_lstm,robust_scaler_LSTM_features,robust_scaler_LSTM_target,correction_lstm_table_temp['actual_raw_predited_ratio'][int(model_case_version_time_steps)-1:].to_numpy()

#this creates the data_sequences(time widows) for the correction LSTM
def correction_data_sequences(data, timesteps): #timesteps means how many days we consider for each block
    X=[]
    y_MPNxP = []
    for i in range(len(data)-timesteps+1):  #how it works: every timesteps (e.g. 10 days) a block is constituted and for each block data and true values are stored
        X.append(data[i:(i+timesteps),:4])
        y_MPNxP.append(data[i+timesteps-1,4])
    return np.array(X),np.array(y_MPNxP)

In [47]:
'''This is the core part, where the the model is defined and trained. In particular, we're using an approach defined by us as 'Multiple Run' because what we do is to iterate the training of our model for a certain amount of times,
until we do not obtain the best one (according to a certain value of course) over all the others. More in details: we train a model and for this model we compute the compound_run_term (that is the value used to eveluate the quality
of the model itself). Of course, at the beginning the first model we obtain is the best one. However, in the next iteration, we train another model (with the same parameters, layers, etc.) and also for this second model we compute
the compound_run_term (to assess its quality). If the compound_run_term of the second model is better ('better' in this case means that is closer to 1) than the best model we've obtained so far (so the first one), then the second
model becomes the new best model. So, we proceed in this way, until either we reach the maximum number of iterations or we obtain a compound_run_term with a certain precision. This approach was born because of an instability problem
that affects our model'''

after_training_predictions = []

for target in range(len(targets)): #for each target
  analytical_parameters = {
    'slope_predicted_calculated_target_ratio_versus_period':[], #slope of the trend line obtained by considering the ratio between predicted values and actual values.
    'intercept_predicted_calculated_target_ratio_versus_period':[], #intercept of the trend line obtained by considering the ratio between predicted values and actual values.
    'end_intercept_predicted_calculated_target_ratio_versus_period':[],
    'rsqr_predicted_calculated_target_ratio_versus_period':[], #r2 of the trend line obtained by considering the ratio between predicted values and actual values.
    'average_tld_predicted_calculated_target_ratio_versus_period':[], #average trend line distance of the trend line obtained by considering the ratio between predicted values and actual values.

    'slope_predicted_versus_calculated_target':[], #slope of the trend line obtained by considering the actual values as x and the predicted values as y.
    'intercept_predicted_versus_calculated_target':[], #intercept of the trend line obtained by considering the actual values as x and the predicted values as y.
    'rsqr_predicted_versus_calculated_target':[], #r2 of the trend line obtained by considering the actual values as x and the predicted values as y.
    'average_tld_predicted_versus_calculated_target':[], #average trend line distance of the trend line obtained by considering the actual values as x and the predicted values as y.

    'normalized_slope_predicted_calculated_target_ratio_versus_period': [],
    'normalized_intercept_predicted_calculated_target_ratio_versus_period':[],
    'normalized_end_intercept_predicted_calculated_target_ratio_versus_period':[],
    'normalized_rsqr_predicted_calculated_target_ratio_versus_period': [],
    'rescaled_normalized_average_tld_predicted_calculated_target_ratio_versus_period': [],

    'normalized_slope_predicted_versus_calculated_target': [],
    'rescaled_normalized_intercept_predicted_versus_calculated_target': [],
    'normalized_rsqr_predicted_versus_calculated_target': [],
    'rescaled_normalized_average_tld_predicted_versus_calculated_target': [],

    'compound_run_term': [],
    'best_run': [],
    'adjusted_compound_run_term':[],
    'padding_correction_factor':[],
    'padding_correction_factor_attenuation':[],
    'absolute_difference_normalized_intercept': [],
    'vertical_padding_correction_factor':[],
    'train_vertical_trend_slope':[]
    }
  
  new_analytical_parameters = {
    'normalized_trend_slope':[],
    'normalized_trend_intercept':[],
    'normalized_trend_end_intercept':[],
    'normalized_trend_rsqr':[],
    'rescaled_normalized_trend_dispersion':[],

    'trend_slope_weighting_exponent':[],
    'trend_intercept_weighting_exponent':[],
    'trend_end_intercept_weighting_exponent':[],
    'trend_rsqr_weighting_exponent':[],
    'trend_dispersion_weighting_exponent':[],

    'adjusted_compound_run_value':[],
    'number_of_runs_performed': [],
    'best_run_number':[],
    'best_model_.h5_file_link':[],
    'best_correction_model_.h5_file_link':[]

    }
  
  '''Inzialization of some values'''
  y_target = y_train #the current target we're considering
  diff = 1e3 #this is used to determine whether the current best model is still the best or not.
  best_run = -1 #this is used to take trace of the best iteration so far
  adjusted_compound_run_term = 1e3 #it is initialized with a very big value just to iterate at least one time the while loop
  iteration = 0 #counter for iterations
  attenuated_padding_value = 1 #initial value for the attenuating padding value
  best_model_adjusted_compound_run_term=0#initial value of the best_model adjusted compound run term
  # last_best_model_run_number=0# initial value for the last best model run number
  # current_best_model_run_number=0#initial value for the current best model run number
  last_best_model_path_h5='dummy.h5'#initial dummy value for the last best model path
  last_best_correction_model_path_h5='correction.h5'

  max_iterations=max(prediction_runs,correction_runs) if prediction_runs!=0 or correction_runs!=0 else 1
  #print('Max iterations: ', max_iterations)

  while abs(adjusted_compound_run_term - 1.0) >= precision and iteration <= max_iterations:
    print('Iteration', iteration, 'for target',targets[target])
    print('Attenuated padding value', attenuated_padding_value)
    print('adjusted_compound_run_term ', adjusted_compound_run_term)

    if prediction_runs==0:
      # default_path_prediction=(best_models_path+targets[target]+'/proceedit '+today+' SPP-'+case+'_20000103-20221130_MPN'+model_case_version_main_target_code+'P_LSTM15_DOHLCAV-10-S-KER-710760-64-100-0.0005-10-'+str(analytical_parametrs)+'L'+str(attenuation_factor)+'_ME - Best model for '+targets[target]+'run_'+str(best_prediction_run)+'.h5')
      default_path_prediction=(best_models_path+targets[target]+'/proceedit 20231230 SPP-AAPL_20000103-20221130_MPN5P_LSTM15_DOHLCAV-10-S-KER-710760-64-100-0.0005-10-11013L0.75_ME - Best model for MPN5Prun_0.h5')
      # default_path_prediction=(best_models_path+targets[target]+'/proceedit 20230826 SPP-^GSPC_20200102-20230118_MPN5P_LSTM-15710760-B64E100L0.0005T8-DOHLCAV-FTEKR_LVSP_LSTM-15710760-B64E100L0.0005T8-PR_11011L0.75_AH - Best model for MPN7Prun_0.h5') #TSLA
      prediction_model=load_model(default_path_prediction,compile=False)
      print(default_path_prediction)

    else:
      if iteration < prediction_runs:
        #print(y_target)
        '''The layers of the model (see case_version_cat Tab)'''
        tf.keras.backend.clear_session()
        prediction_model=tf.keras.Sequential()
        prediction_model.add(LSTM(2**twoexp_nodes_number_layer_1,input_shape=input_shape,return_sequences=True))
        prediction_model.add(LSTM(2**twoexp_nodes_number_layer_2, return_sequences=True))
        prediction_model.add(LSTM(2**twoexp_nodes_number_layer_3))
        prediction_model.add(Dense(2**twoexp_nodes_number_layer_4))
        prediction_model.add(Dense(int(model_case_version_main_target_code)+1))
        prediction_model.compile(optimizer = optimizers.Adam(learning_rate = lr), loss = custom_loss_function(attenuated_padding_value))

        '''Training of the model'''
        print('\n')
        print(' ----------- Training prediction model started -----------')
        print('\n')
        prediction_model.fit(X_train, y_target, batch_size=batch, epochs=1, verbose=2)

    '''Computation of the predictions'''
    train_predictions=prediction_model.predict(X_train) #predictions for training data
    train_predictions = train_robust_scaler_target.inverse_transform(train_predictions) #convert prediction first by inverting the Robust scaler transformation and then the e_logarithmic one.
    train_predictions = np.exp(train_predictions)
    train_prediction = [a[:1] for a in train_predictions]
    train_array = np.concatenate(train_prediction)

    multiple_run_predictions = train_array
    actual = actuals_cols[0][:len(multiple_run_predictions)]
    pred_actual_vs_date_ratio = train_array / actual # why date?
    
    _, _, _, pred_actual_vs_date_ratio_dispersion= sir_parameters(positions_day_number_train,pred_actual_vs_date_ratio)
    print("pred_actual_vs_date_ratio_dispersion",pred_actual_vs_date_ratio_dispersion)

    _, _, _, pred_vs_actual_dispersion= sir_parameters(actual,train_array)
    print("pred_actual_dispersion",pred_vs_actual_dispersion)

    '''implementation of the correction phases for the LSTM corrected_predicted_targets'''
    '''PHASE 1 CORRECTION'''
    #vertical_correction
    raw_predicted_targets = train_array
    train_raw_targets_ratio = raw_predicted_targets/actual

    if apply_vertical_swing_corr:
      print('\n')
      print(' ----------- Training correction model started -----------')
      print('\n')
      train_raw_trend_slope, train_raw_trend_intercept, _, train_raw_trend_dispersion= sir_parameters(positions_day_number_train,train_raw_targets_ratio)
      print('train_raw_trend_slope:',train_raw_trend_slope)
      print('train_raw_trend_intercept:',train_raw_trend_intercept)
      print('train_raw_trend_dispersion',train_raw_trend_dispersion)

      vertical_padding_correction_factor = 1/train_raw_trend_intercept
      print("vertical_padding_correction_factor:",vertical_padding_correction_factor)
      corrected_train_raw_targets_ratios = train_raw_targets_ratio * vertical_padding_correction_factor
      print("corrected_train_raw_targets_ratios", corrected_train_raw_targets_ratios[:10])
      train_vertical_corrected_targets = corrected_train_raw_targets_ratios * actual#this is the formula for vertical padding
      print("back_vertical_corrected_targets", train_vertical_corrected_targets[:10])

      swing_padding_correction_factors = []
      train_vertical_targets_ratios=train_vertical_corrected_targets/actual
      train_vertical_trend_slope, train_vertical_trend_intercept, _, train_vertical_trend_dispersion= sir_parameters(positions_day_number_train, train_vertical_targets_ratios)

      for i in positions_day_number_train:
        swing_padding_correction_factor = 1 / (1 + i * train_vertical_trend_slope)
        swing_padding_correction_factors.append(swing_padding_correction_factor)

      print('swing_padding_correction_factors',swing_padding_correction_factors[:10])
      print('period_day_numbers', positions_day_number_train[:10])

      swing_targets_ratios = train_raw_targets_ratio* vertical_padding_correction_factor*swing_padding_correction_factors
      swing_predicted_targets = swing_targets_ratios * actual

      print('swing_predicted_targets: ', swing_predicted_targets[:10])

    else:
      swing_predicted_targets=raw_predicted_targets

    '''implementation of correction lstm'''
    if True:
      volume = df_actual['VTCP']
      correction_lstm_table,lstm_robust_feat,lstm_robust_target,actual_correct=preprare_correction_lstm_table_train(positions_day_number_train,swing_predicted_targets,volume,actual)
      correction_train_data=correction_lstm_table
      X_correction_train, y_correction_train =correction_data_sequences(correction_train_data, int(model_case_version_time_steps))
      #print(correction_train_data)
      print(X_correction_train)
      # sys.exit()
      correction_input_shape=((X_correction_train).shape[1],(X_correction_train).shape[2])
      print(correction_input_shape)
      if correction_runs==0:
        # default_path_correction=(best_models_path+targets[target]+'/proceedit '+today+' SPP-'+case+'_20000103-20221130_MPN'+model_case_version_main_target_code+'P_LSTM15_DOHLCAV-10-S-KER-710760-64-100-0.0005-10-'+str(analytical_parametrs)+'L'+str(attenuation_factor)+'_ME - Best model for '+targets[target]+'run_'+str(best_correction_run)+'correction_model.h5')
        default_path_correction=(best_models_path+targets[target]+'/proceedit 20230827 SPP-TSLA_20200102-20230531_MPN5P_LSTM-15710760-B64E100L0.0005T8-DOHLCAV-FTEKR_LVSP_LSTM-15710760-B64E100L0.0005T8-PR_11011L0.75_DM - Best model for MPN5Prun_5correction_model.h5') #TSLA
        correction_model=load_model(default_path_correction,compile=False)
      else:
        if  iteration < correction_runs:
          '''correction lstm implementation'''
          correction_model= tf.keras.Sequential()
          correction_model.add(LSTM(2**twoexp_nodes_number_layer_1,input_shape=correction_input_shape,return_sequences=True))
          correction_model.add(LSTM(2**twoexp_nodes_number_layer_2, return_sequences=True))
          correction_model.add(LSTM(2**twoexp_nodes_number_layer_3))
          correction_model.add(Dense(2**twoexp_nodes_number_layer_4))
          correction_model.add(Dense(2**twoexp_nodes_number_layer_5))
          correction_model.compile(optimizer = optimizers.Adam(learning_rate =correction_lr), loss = custom_loss_function(attenuated_padding_value))
          correction_model.fit(X_correction_train, y_correction_train, batch_size=correction_batch, epochs=1, verbose=2)

      LSTM_corrections=correction_model.predict(X_correction_train)
      print("raw_LSTM_corrections",LSTM_corrections[:10])
      LSTM_corrections=lstm_robust_target.inverse_transform(LSTM_corrections.reshape(-1,1))
      print("transformed_LSTM_corrections",LSTM_corrections[:10])
      LSTM_corrections=LSTM_corrections.reshape(1,-1)[0]
      # actual_corrections= correction_lstm_table['actual_raw_predited_ratio'][int(model_case_version_time_steps)-1:].to_numpy()
      compare_corrections=lstm_robust_target.inverse_transform(correction_lstm_table[:,2][int(model_case_version_time_steps)-1 + int(model_case_version_main_target_code):].reshape(1,-1))
      actual_corrections=actual_correct
      corrected_predicted_targets=np.multiply(swing_predicted_targets[int(model_case_version_time_steps)-1 + (2*int(model_case_version_main_target_code)):],LSTM_corrections)
      print("LSTM_corrections",LSTM_corrections[:10])
      print("raw_predicted_targets_for_LSTM",swing_predicted_targets[int(model_case_version_time_steps)-1 + (2*int(model_case_version_main_target_code)):][:10])
      print("corrected_predicted_targets",corrected_predicted_targets[:10])
      _, _, _, correction_dispersion= sir_parameters(actual_corrections[int(model_case_version_main_target_code):],LSTM_corrections)
      print("pred_vs_actual_correction_dispersion",correction_dispersion)
      # plt.figure(figsize=(10,10))
      # a, b = np.polyfit(actual_corrections,LSTM_corrections, 1)
      # plt.scatter(actual_corrections, LSTM_corrections, color='purple')
      # plt.plot(actual_corrections, a*actual_corrections+b,color='steelblue', linewidth=2)
      # plt.title('predicted_vs_actual_correction_plot')
      # plt.show()
      period_day_number_temp=positions_day_number_train[int(model_case_version_time_steps)-1 + (2*int(model_case_version_main_target_code)):]
      period_day_number=period_day_number_temp-period_day_number_temp[0]+1
      actual_moving_average=actual[int(model_case_version_time_steps)-1 + (2*int(model_case_version_main_target_code)):]
    else:
      period_day_number=positions_day_number_train
      corrected_predicted_targets=swing_predicted_targets
      actual_moving_average=actual

    if past_corr_train_stat:
      ''' Implementation of last padding correction'''
      last_padding_correction_factors = []
      last_padding_correction_factors.append(1)
      last_padding_correction_factors.append(1)
      last_padding_correction_factors.append(1)
      last_padding_correction_factors.append(1)
      # last_padding_correction_factors.append(1)
      # last_padding_correction_factors.append(1)
      for i in range(4,len(period_day_number)):
        last_padding_correction_factors.append(((actual_moving_average[i-1]+actual_moving_average[i-2]+actual_moving_average[i-3]+actual_moving_average[i-4])/4)/((corrected_predicted_targets[i-1]+corrected_predicted_targets[i-2]+corrected_predicted_targets[i-3]+corrected_predicted_targets[i-4])/4))

      print(len(last_padding_correction_factors))
      # initial_last_ratio=swing_targets_ratios
      # initial_last_ratio=LSTM_corrections
      initial_last_ratio=corrected_predicted_targets/actual_moving_average

      last_targets_ratios = initial_last_ratio * last_padding_correction_factors
      last_slope,last_start_intercept,_,last_dispersion=sir_parameters(period_day_number,last_targets_ratios)
      last_predicted_targets = last_targets_ratios*actual_moving_average
      print('last_padding_correction_factors:',last_padding_correction_factors)
      print('last_targets_ratios:',last_targets_ratios[:10])
      print('last_slope:', last_slope)
      print('last_start_intercept:',last_start_intercept)
      print('last_dispersion:',last_dispersion)
      print('last_predicted_targets:',last_predicted_targets[:10])
      print()

    else:
      last_predicted_targets=corrected_predicted_targets

    iteration += 1


Iteration 0 for target MPN5P
Attenuated padding value 1
adjusted_compound_run_term  1000.0


2024-01-16 09:19:09.316006: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 09:19:09.316311: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 09:19:09.316523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-



 ----------- Training prediction model started -----------




2024-01-16 09:19:25.374580: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.00MiB (rounded to 16777216)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-16 09:19:25.374624: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-01-16 09:19:25.374636: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 23, Chunks in use: 23. 5.8KiB allocated for chunks. 5.8KiB in use in bin. 376B client-requested in use in bin.
2024-01-16 09:19:25.374645: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-01-16 09:19:25.374653: I external/local_ts

ResourceExhaustedError: in user code:

    File "/home/ubuntu/LTM_Bot/LTM-Long-short_Term_Memory.Bot-API_0.00/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/ubuntu/LTM_Bot/LTM-Long-short_Term_Memory.Bot-API_0.00/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ubuntu/LTM_Bot/LTM-Long-short_Term_Memory.Bot-API_0.00/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/ubuntu/LTM_Bot/LTM-Long-short_Term_Memory.Bot-API_0.00/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/ubuntu/LTM_Bot/LTM-Long-short_Term_Memory.Bot-API_0.00/e